In [59]:
%load_ext autoreload
%autoreload 2

from utils.hamiltonians import Max3satHamiltonian
from utils.qaoa import QAOA
from qiskit import transpile
from qiskit.circuit import QuantumCircuit, Parameter

hamiltonian = Max3satHamiltonian("./instances/uf20-01.cnf")

print(hamiltonian.formula.nv, hamiltonian.formula.clauses)
print(hamiltonian.get_pauli_list(), len(hamiltonian.get_pauli_list()))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
20 [[4, -18, 19], [3, 18, -5], [-5, -8, -15], [-20, 7, -16], [10, -13, -7], [-12, -9, 17], [17, 19, 5], [-16, 9, 15], [11, -5, -14], [18, -10, 13], [-3, 11, 12], [-6, -17, -8], [-18, 14, 1], [-19, -15, 10], [12, 18, -19], [-8, 4, 7], [-8, -9, 4], [7, 17, -15], [12, -7, -14], [-10, -11, 8], [2, -15, -11], [9, 6, 1], [-11, 20, -17], [9, -15, 13], [12, -7, -17], [-18, -2, 20], [20, 12, 4], [19, 11, 14], [-16, 18, -4], [-1, -17, -19], [-13, 15, 10], [-12, -14, -13], [12, -14, -7], [-7, 16, 10], [6, 10, 7], [20, 14, -16], [-19, 17, 11], [-7, 1, -20], [-5, 12, 15], [-4, -9, -13], [12, -11, -7], [-5, 19, -8], [1, 16, 17], [20, -14, -15], [13, -4, 10], [14, 7, 10], [-5, 9, 20], [10, 1, -19], [-16, -15, -1], [16, 3, -11], [-15, -10, 4], [4, -15, -3], [-10, -16, 11], [-8, 12, -5], [14, -6, 12], [1, 6, 11], [-13, -5, -1], [-7, -2, 12], [1, -20, 19], [-2, -13, -8], [15, 18, 4], [-11, 14, 9], [-6, -15, -2], [5, 

In [63]:
basis_gates = ["rx", "rz", "x", "y", "z", "h", "id", "cz"]
depth = 1
qaoa = QAOA(hamiltonian)
parameter = Parameter("phi")
qaoa_circuit, cost_params, mixer_params = qaoa.naive_qaoa_circuit(depth)
bound_circuit = qaoa_circuit.assign_parameters({cost_params: [1 for param in cost_params], mixer_params: [1 for param in mixer_params]})
transpiled_circuit = transpile(bound_circuit, basis_gates=basis_gates, optimization_level=3)

print(transpiled_circuit.depth())

233


In [18]:
from qiskit.circuit import QuantumCircuit
from qiskit.quantum_info import Operator, SparsePauliOp
import numpy as np

circuit = QuantumCircuit(3)
circuit.ccx(0, 1, 2)
circuit.rz(np.pi, 2)
circuit.cx(2, 1)
circuit.rz(np.pi, 1)
circuit.cx(2, 1)
circuit.ccx(0, 1, 2)
operator = SparsePauliOp.from_operator(Operator(circuit).data)

print(operator)
print(circuit.draw())

SparsePauliOp(['IZI'],
              coeffs=[-1.+0.j])
                                           
q_0: ──■────────────────────────────────■──
       │           ┌───┐┌───────┐┌───┐  │  
q_1: ──■───────────┤ X ├┤ Rz(π) ├┤ X ├──■──
     ┌─┴─┐┌───────┐└─┬─┘└───────┘└─┬─┘┌─┴─┐
q_2: ┤ X ├┤ Rz(π) ├──■─────────────■──┤ X ├
     └───┘└───────┘                   └───┘
